In [1]:
# kaggle.json
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [175]:
!pip install kaggle
!kaggle competitions download -c 'demand-forecasting-in-retail'
!unzip train.csv.zip

test.csv: Skipping, found more recently modified local copy (use --force to force download)
STORE_LOCATION.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [344]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Читаем все датасеты:

In [345]:
df_train = pd.read_csv('./train.csv')
df_train.rename(columns={"Unnamed: 0": "id"}, inplace=True)
df_stores = pd.read_csv('./STORE_LOCATION.csv', delimiter=";", index_col=None)
df_test = pd.read_csv('./test.csv')
temp = pd.read_csv('./sample_submission.csv', index_col=None)
temp = temp.id.values

Заполним недостающие значения самой популярной категорией:

In [346]:
df_train.PROMO1_FLAG = df_train.PROMO1_FLAG.fillna(0)
df_train.AUTORIZATION_FLAG = df_train.AUTORIZATION_FLAG.fillna(1)

Выкинем неинформативные столбцы:

In [347]:
df_stores = df_stores.drop(
    columns=[
        "STORE_LOCATION_ATTRIB17_hashing",
        "STORE_LOCATION_ATTRIB18_hashing",
        "STORE_LOCATION_ATTRIB19_hashing",
        "STORE_LOCATION_ATTRIB20_hashing",
        "STORE_LOCATION_ATTRIB21_hashing",
        "STORE_LOCATION_LVL_RK1",
        "STORE_CLOSURE_DTTM",
        "STORE_LOCATION_ADK_hashing"
    ]
)

In [348]:
df_stores

,STORE_LOCATION_RK,STORE_LOCATION_LVL_RK4,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK2,STORE_OPEN_DTTM,STORE_LOCATION_ATTRIB1_hashing,STORE_LOCATION_ATTRIB2_hashing,STORE_LOCATION_ATTRIB3_hashing,STORE_LOCATION_ATTRIB4_hashing,STORE_LOCATION_ATTRIB5_hashing,STORE_LOCATION_ATTRIB6_hashing,STORE_LOCATION_ATTRIB7_hashing,STORE_LOCATION_ATTRIB8_hashing,STORE_LOCATION_ATTRIB9_hashing,STORE_LOCATION_ATTRIB10_hashing,STORE_LOCATION_ATTRIB11_hashing,STORE_LOCATION_ATTRIB12_hashing,STORE_LOCATION_ATTRIB13_hashing,STORE_LOCATION_ATTRIB14_hashing,STORE_LOCATION_ATTRIB15_hashing,STORE_LOCATION_ATTRIB16_hashing
0,277,124,124,19,15Mar2018:14:08:08,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,9B9B954D2D5B0F1C5B0B129DF8DED631,691ED43F4AF65A71A6FB21210CBE5109,2356B83B3AE123A290CB245A8F8526DA,75300434BE16A57FD386B22D19B289D5,CB889E260FC7B06777314137C2FB5198,1FAA6C1A11273624AB0AB65A2B4088E8,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3
1,278,124,124,19,15Mar2018:14:08:08,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,81C7FB4E26CB45F0745DF4A59EB6CD9E,691ED43F4AF65A71A6FB21210CBE5109,2356B83B3AE123A290CB245A8F8526DA,75300434BE16A57FD386B22D19B289D5,CB889E260FC7B06777314137C2FB5198,1FAA6C1A11273624AB0AB65A2B4088E8,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,0A7D98B3F3BA0A5BC89F693FB146FE40,3F0DA22658EF921D0F500BFDC5471DB3
2,279,183,183,20,15Mar2018:14:08:08,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,9B9B954D2D5B0F1C5B0B129DF8DED631,22301E873A3C0BAD5C4C0A912439DB4B,2356B83B3AE123A290CB245A8F8526DA,75300434BE16A57FD386B22D19B289D5,CB889E260FC7B06777314137C2FB5198,89BE5E5DCB4CC7381644A39851FE862B,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3
3,280,211,211,12,15Mar2018:14:08:08,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,B5A368A9689D233843CB7A34709BD45B,271329C6DEA8D933600B7F11344D3D42,5987D1803F8B590AF4B65DC273428555,75300434BE16A57FD386B22D19B289D5,B4A0AD777A28164B0EBB338E14AB134A,B21DA7BB444B2B2BD5D26E5E5C06A76B,B2581C791DBB8894BFDCFB66B60A17ED,2C3957E2A91B1D2BEE334248B387E402,67B1193B31375E3C959BB5246989D0CA,67B1193B31375E3C959BB5246989D0CA,17AC9AE74D16FBC665FFB4CFC2DF24F5,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3
4,281,221,221,3,15Mar2018:14:08:08,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,D07B12CA0F54B053829C81488035F3E2,F4A8074414856FD8DA2FE9597A177859,FA65AC5654DB757AF153BC92AB4B1A15,75300434BE16A57FD386B22D19B289D5,1E2F73498E9CF154EA2175CC36A1133A,3065B24375F2271B030ED39812336A10,B2581C791DBB8894BFDCFB66B60A17ED,2C3957E2A91B1D2BEE334248B387E402,67B1193B31375E3C959BB5246989D0CA,67B1193B31375E3C959BB5246989D0CA,17AC9AE74D16FBC665FFB4CFC2DF24F5,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,1482,227,227,2,30Nov2019:03:40:21,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,FB46D3D3CE99B92582D8D6B52CB0DFF8,07C1897B8BCE9023547A9D067C58F4B5,FA65AC5654DB757AF153BC92AB4B1A15,75300434BE16A57FD386B22D19B289D5,5AC69945824913C1D3E6772447B65411,FF83472786DB3B25D48ABD61502E1D91,D2C50D70FBCF0A56020B70626BF2FB2C,1403C212BBBAFF12456F25AA5F40B132,222236EE6D50563687CE7BBC6A1D6388,222236EE6D50563687CE7BBC6A1D6388,E393D33ACC3FB49899A4E1F1707ACC7C,B70EA5F8E1DD1E21F7

Объединим данные по магазинам с остальными данными:

In [349]:
df_train = df_train.merge(df_stores, how='left', left_on='store_location_rk', right_on='STORE_LOCATION_RK')

In [350]:
df_train.head()

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,STORE_LOCATION_RK,STORE_LOCATION_LVL_RK4,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK2,STORE_OPEN_DTTM,STORE_LOCATION_ATTRIB1_hashing,STORE_LOCATION_ATTRIB2_hashing,STORE_LOCATION_ATTRIB3_hashing,STORE_LOCATION_ATTRIB4_hashing,STORE_LOCATION_ATTRIB5_hashing,STORE_LOCATION_ATTRIB6_hashing,STORE_LOCATION_ATTRIB7_hashing,STORE_LOCATION_ATTRIB8_hashing,STORE_LOCATION_ATTRIB9_hashing,STORE_LOCATION_ATTRIB10_hashing,STORE_LOCATION_ATTRIB11_hashing,STORE_LOCATION_ATTRIB12_hashing,STORE_LOCATION_ATTRIB13_hashing,STORE_LOCATION_ATTRIB14_hashing,STORE_LOCATION_ATTRIB15_hashing,STORE_LOCATION_ATTRIB16_hashing
0,0,40369,309,2016-12-19,29.0,0.0,NaN,NaN,NaN,NaN,1.0,309,203,203,10,15Mar2018:14:08:08,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,B5A368A9689D233843CB7A34709BD45B,1D870936A2C35AA44CB0CF9B6210FB74,F5126A9BDB60B4C8F3B1680AF38B33B9,75300434BE16A57FD386B22D19B289D5,BC0EB006F36BC82F8E165D6D4E95F207,F2D475B39396781ED7F2D76E648AFAE4,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3
1,1,40370,309,2016-12-19,64.0,0.0,NaN,NaN,NaN,NaN,1.0,309,203,203,10,15Mar2018:14:08:08,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,B5A368A9689D233843CB7A34709BD45B,1D870936A2C35AA44CB0CF9B6210FB74,F5126A9BDB60B4C8F3B1680AF38B33B9,75300434BE16A57FD386B22D19B289D5,BC0EB006F36BC82F8E165D6D4E95F207,F2D475B39396781ED7F2D76E648AFAE4,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3
2,2,40372,309,2016-12-19,32.0,0.0,NaN,NaN,NaN,NaN,1.0,309,203,203,10,15Mar2018:14:08:08,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,B5A368A9689D233843CB7A34709BD45B,1D870936A2C35AA44CB0CF9B6210FB74,F5126A9BDB60B4C8F3B1680AF38B33B9,75300434BE16A57FD386B22D19B289D5,BC0EB006F36BC82F8E165D6D4E95F207,F2D475B39396781ED7F2D76E648AFAE4,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3
3,3,40373,309,2016-12-19,10.0,0.0,NaN,NaN,NaN,NaN,1.0,309,203,203,10,15Mar2018:14:08:08,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,B5A368A9689D233843CB7A34709BD45B,1D870936A2C35AA44CB0CF9B6210FB74,F5126A9BDB60B4C8F3B1680AF38B33B9,75300434BE16A57FD386B22D19B289D5,BC0EB006F36BC82F8E165D6D4E95F207,F2D475B39396781ED7F2D76E648AFAE4,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3
4,4,46272,309,2016-12-19,15.0,0.0,NaN,NaN,NaN,NaN,1.0,309,203,203,10,15Mar2018:14:08:08,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,B5A368A9689D233843CB7A34709BD45B,1D870936A2C35AA44CB0CF9B6210FB74,F5126A9BDB60B4C8F3B1680AF38B33B9,75300434BE16A57FD386B22D19B289D5,BC0EB006F36BC82F8E165D6D4E95F207,F2D475B39396781ED7F2D76E648AFAE4,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3


In [351]:
df_train.describe()

,id,product_rk,store_location_rk,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,STORE_LOCATION_RK,STORE_LOCATION_LVL_RK4,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK2
count,35344.000000,35344.000000,35344.000000,34144.000000,35344.000000,35159.0,34217.000000,34212.000000,35159.0,35344.000000,35344.000000,35344.00000,35344.00000,35344.0
mean,17766.554012,49253.732232,844.240154,12.245636,0.205353,0.0,1167.679357,1155.778351,0.0,0.908160,844.240154,134.23226,134.23226,10.0
std,10258.040738,19145.064867,333.229160,32.604642,0.432514,0.0,1046.828551,1057.912830,0.0,0.288804,333.229160,82.72710,82.72710,0.0
min,0.000000,40369.000000,309.000000,0.000000,0.000000,0.0,49.000000,8.647059,0.0,0.000000,309.000000,25.00000,25.00000,10.0
25%,8881.750000,40370.000000,535.000000,2.000000,0.000000,0.0,284.290000,199.000000,0.0,1.000000,535.000000,39.00000,39.00000,10.0
50%,17770.500000,40372.000000,862.000000,6.000000,0.000000,0.0,1000.000000,1000.000000,0.0,1.000000,862.000000,189.00000,189.00000,10.0
75%,26647.250000,46272.000000,1173.000000,12.000000,0.000000,0.0,2000.000000,2000.000000,0.0,1.000000,1173.000000,203.00000,203.00000,10.0
max,35541.000000,96212.000000,1380.000000,1160.000000,2.000000,0.0,3000.000000,3000.000000,0.0,1.000000,1380.000000,258.00000,258.00000,10.0


In [352]:
 df_train.isna().sum()

id                                    0
product_rk                            0
store_location_rk                     0
period_start_dt                       0
demand                             1200
PROMO1_FLAG                           0
PROMO2_FLAG                         185
PRICE_REGULAR                      1127
PRICE_AFTER_DISC                   1132
NUM_CONSULTANT                      185
AUTORIZATION_FLAG                     0
STORE_LOCATION_RK                     0
STORE_LOCATION_LVL_RK4                0
STORE_LOCATION_LVL_RK3                0
STORE_LOCATION_LVL_RK2                0
STORE_OPEN_DTTM                       0
STORE_LOCATION_ATTRIB1_hashing        0
STORE_LOCATION_ATTRIB2_hashing        0
STORE_LOCATION_ATTRIB3_hashing        0
STORE_LOCATION_ATTRIB4_hashing        0
STORE_LOCATION_ATTRIB5_hashing        0
STORE_LOCATION_ATTRIB6_hashing        0
STORE_LOCATION_ATTRIB7_hashing        0
STORE_LOCATION_ATTRIB8_hashing        0
STORE_LOCATION_ATTRIB9_hashing        0


In [353]:
print(df_train.NUM_CONSULTANT.value_counts())
print(df_train.PROMO2_FLAG.value_counts())
print(df_train.PROMO1_FLAG.value_counts())
print(df_train.store_location_rk.value_counts())
print(df_train.product_rk.value_counts())
print(df_train.period_start_dt.value_counts())
print(df_train.demand.value_counts())

0.0    35159
Name: NUM_CONSULTANT, dtype: int64
0.0    35159
Name: PROMO2_FLAG, dtype: int64
0.0    28508
1.0     6414
2.0      422
Name: PROMO1_FLAG, dtype: int64
798     924
1034    924
1281    924
657     924
764     924
540     924
1202    924
517     924
1005    924
425     924
533     924
525     923
555     923
1162    923
862     923
874     923
453     923
644     923
355     923
866     923
1191    923
1185    923
1080    923
562     923
637     923
317     923
1173    923
380     923
1143    922
1203    922
535     922
1079    922
557     921
504     919
1326    859
1328    726
1316    719
1363    669
1347    648
1380    325
309      15
Name: store_location_rk, dtype: int64
40373    6074
40369    6074
40370    6074
40372    6073
46272    6068
96212    4981
Name: product_rk, dtype: int64
2019-07-22    240
2019-02-18    240
2019-06-24    240
2019-11-11    240
2019-09-09    240
             ... 
2017-01-30    170
2017-02-13    170
2017-01-09    170
2017-01-16    170
2017-02-06 

Опять удаляем неинформативные столбцы:

In [354]:
df_train.drop('NUM_CONSULTANT', axis=1, inplace=True)
df_train.drop('PROMO2_FLAG', axis=1, inplace=True)
promo_df = pd.get_dummies(df_train.PROMO1_FLAG, prefix='PROMO1_FLAG')
#df_train.drop('PROMO1_FLAG', axis=1, inplace=True)
df_train = pd.concat([df_train, promo_df], axis=1)

In [355]:
df_test['is_test'] = np.repeat(True, df_test.shape[0])
df_train['is_test'] = np.repeat(False, df_train.shape[0])
df_total = pd.concat([df_train, df_test])
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36748 entries, 0 to 1403
Data columns (total 34 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               36748 non-null  int64  
 1   product_rk                       36748 non-null  int64  
 2   store_location_rk                36748 non-null  int64  
 3   period_start_dt                  36748 non-null  object 
 4   demand                           34144 non-null  float64
 5   PROMO1_FLAG                      35344 non-null  float64
 6   PRICE_REGULAR                    34217 non-null  float64
 7   PRICE_AFTER_DISC                 34212 non-null  float64
 8   AUTORIZATION_FLAG                35344 non-null  float64
 9   STORE_LOCATION_RK                35344 non-null  float64
 10  STORE_LOCATION_LVL_RK4           35344 non-null  float64
 11  STORE_LOCATION_LVL_RK3           35344 non-null  float64
 12  STORE_LOCATION_LVL_

Делаем категорийные и численные признаки для дат:

In [356]:
df_total["period_start_dt"] = pd.to_datetime(df_total["period_start_dt"])

weekday_df = pd.get_dummies(df_total.period_start_dt.dt.weekday, prefix='weekday')
month_df = pd.get_dummies(df_total.period_start_dt.dt.month, prefix='month')
year_df = pd.get_dummies(df_total.period_start_dt.dt.year, prefix='year')

df_total = pd.concat([weekday_df, month_df, year_df, df_total], axis=1)
df_total["day"] = df_total["period_start_dt"].dt.day
df_total["month"] = df_total["period_start_dt"].dt.month
df_total["day_year"] = df_total["period_start_dt"].dt.dayofyear
df_total["weekday"] = df_total["period_start_dt"].dt.weekday

df_total = df_total.drop(columns=["period_start_dt"])

In [357]:
print(df_total.columns)

Index(['weekday_0', 'weekday_1', 'weekday_3', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12', 'year_2016', 'year_2017',
       'year_2018', 'year_2019', 'year_2020', 'id', 'product_rk',
       'store_location_rk', 'demand', 'PROMO1_FLAG', 'PRICE_REGULAR',
       'PRICE_AFTER_DISC', 'AUTORIZATION_FLAG', 'STORE_LOCATION_RK',
       'STORE_LOCATION_LVL_RK4', 'STORE_LOCATION_LVL_RK3',
       'STORE_LOCATION_LVL_RK2', 'STORE_OPEN_DTTM',
       'STORE_LOCATION_ATTRIB1_hashing', 'STORE_LOCATION_ATTRIB2_hashing',
       'STORE_LOCATION_ATTRIB3_hashing', 'STORE_LOCATION_ATTRIB4_hashing',
       'STORE_LOCATION_ATTRIB5_hashing', 'STORE_LOCATION_ATTRIB6_hashing',
       'STORE_LOCATION_ATTRIB7_hashing', 'STORE_LOCATION_ATTRIB8_hashing',
       'STORE_LOCATION_ATTRIB9_hashing', 'STORE_LOCATION_ATTRIB10_hashing',
       'STORE_LOCATION_ATTRIB11_hashing', 'STORE_LOCATION_ATTRIB12_hashing',
       'ST

Нормализуем данные и создаем категории:

In [358]:
scaler = MinMaxScaler(feature_range=(0,1))
cols_to_scale = [col for col in df_total.columns if 'demand' not in col and 'id' not in col and 'STORE_' not in col]
scaled_cols = scaler.fit_transform(df_total[cols_to_scale])

cols_to_category = [col for col in df_total.columns if 'STORE_' in col]
df_total = pd.get_dummies(df_total, columns=cols_to_category)

df_total[cols_to_scale] = scaled_cols
df_total.PRICE_REGULAR = df_total.PRICE_REGULAR.fillna(df_total.PRICE_REGULAR.mean())
df_total.PRICE_AFTER_DISC = df_total.PRICE_AFTER_DISC.fillna(df_total.PRICE_AFTER_DISC.mean())

df_train = df_total[df_total['is_test'] == False].drop('is_test', axis=1)
df_test = df_total[df_total['is_test'] == True].drop('is_test', axis=1)

In [359]:
 df_train

,weekday_0,weekday_1,weekday_3,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2016,year_2017,year_2018,year_2019,year_2020,id,product_rk,store_location_rk,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,PROMO1_FLAG_0.0,PROMO1_FLAG_1.0,PROMO1_FLAG_2.0,day,month,day_year,weekday,STORE_LOCATION_RK_309.0,STORE_LOCATION_RK_317.0,STORE_LOCATION_RK_355.0,STORE_LOCATION_RK_380.0,STORE_LOCATION_RK_425.0,...,STORE_LOCATION_ATTRIB1_hashing_B4E5DABD7AF605C64E1CEF8BE96EDEF0,STORE_LOCATION_ATTRIB2_hashing_1B208496FEF79DB5BE9F374BF65865E6,STORE_LOCATION_ATTRIB2_hashing_FE26708E98829533D1F2877AC0D06DAC,STORE_LOCATION_ATTRIB3_hashing_00BADC096F516C72A81763F8CC45E4C6,STORE_LOCATION_ATTRIB3_hashing_14630927ADE80C10BF2E754912A354BD,STORE_LOCATION_ATTRIB3_hashing_39ED0D2B753268FD16E042087A244FDC,STORE_LOCATION_ATTRIB3_hashing_5240EF23D626CBAC7AA7303460206F3A,STORE_LOCATION_ATTRIB3_hashing_5612080FCCC53BDC19C50D0FD9BF5DBA,STORE_LOCATION_ATTRIB3_hashing_78B6B1307FBF3AC2589938DD7ADC39D3,STORE_LOCATION_ATTRIB3_hashing_81C7FB4E26CB45F0745DF4A59EB6CD9E,STORE_LOCATION_ATTRIB3_hashing_9B9B954D2D5B0F1C5B0B129DF8DED631,STORE_LOCATION_ATTRIB3_hashing_B5A368A9689D233843CB7A34709BD45B,STORE_LOCATION_ATTRIB3_hashing_D07B12CA0F54B053829C81488035F3E2,STORE_LOCATION_ATTRIB3_hashing_FB46D3D3CE99B92582D8D6B52CB0DFF8,STORE_LOCATION_ATTRIB4_hashing_04E5A06E3993F6C9A4BFDEF278636ED5,STORE_LOCATION_ATTRIB4_hashing_1D870936A2C35AA44CB0CF9B6210FB74,STORE_LOCATION_ATTRIB4_hashing_9DE874E955029301F76E51E6FAE889BC,STORE_LOCATION_ATTRIB5_hashing_F5126A9BDB60B4C8F3B1680AF38B33B9,STORE_LOCATION_ATTRIB6_hashing_75300434BE16A57FD386B22D19B289D5,STORE_LOCATION_ATTRIB7_hashing_BC0EB006F36BC82F8E165D6D4E95F207,STORE_LOCATION_ATTRIB8_hashing_19517FD4F1E15806B405B9D730EEADCE,STORE_LOCATION_ATTRIB8_hashing_9BE4F7DC711A2F182743CC0C7DB1EA0C,STORE_LOCATION_ATTRIB8_hashing_CB5A0648A9D14B22A85E039557A09205,STORE_LOCATION_ATTRIB8_hashing_F2D475B39396781ED7F2D76E648AFAE4,STORE_LOCATION_ATTRIB9_hashing_B2581C791DBB8894BFDCFB66B60A17ED,STORE_LOCATION_ATTRIB9_hashing_D2C50D70FBCF0A56020B70626BF2FB2C,STORE_LOCATION_ATTRIB10_hashing_1403C212BBBAFF12456F25AA5F40B132,STORE_LOCATION_ATTRIB10_hashing_2C3957E2A91B1D2BEE334248B387E402,STORE_LOCATION_ATTRIB10_hashing_3430A133D252AE6FF57ECB5DFEEA50CD,STORE_LOCATION_ATTRIB10_hashing_410E87854BCC80640DBEFAB137107251,STORE_LOCATION_ATTRIB11_hashing_67B1193B31375E3C959BB5246989D0CA,STORE_LOCATION_ATTRIB11_hashing_A067F6FD038AB399337304ED26250C88,STORE_LOCATION_ATTRIB12_hashing_67B1193B31375E3C959BB5246989D0CA,STORE_LOCATION_ATTRIB12_hashing_6BA1D912365C006B003750C17E810A8C,STORE_LOCATION_ATTRIB13_hashing_17AC9AE74D16FBC665FFB4CFC2DF24F5,STORE_LOCATION_ATTRIB13_hashing_6BA1D912365C006B003750C17E810A8C,STORE_LOCATION_ATTRIB14_hashing_B70EA5F8E1DD1E21F7485AADD433620F,STORE_LOCATION_ATTRIB15_hashing_0A7D98B3F3BA0A5BC89F693FB146FE40,STORE_LOCATION_ATTRIB15_hashing_FC5FD090934CE685C1C12E5E0311F1D4,STORE_LOCATION_ATTRIB16_hashing_3F0DA22658EF921D0F500BFDC5471DB3
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,0.000000,0.0,29.0,0.0,0.379085,0.383482,1.0,1.0,0.0,0.0,0.600000,1.0,0.969780,0.0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1,0.000018,0.0,64.0,0.0,0.379085,0.383482,1.0,1.0,0.0,0.0,0.600000,1.0,0.969780,0.0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2,0.000054,0.0,32.0,0.0,0.379085,0.383482,1.0,1.0,0.0,0.0,0.600000,1.0,0.969780,0.0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3,0.000072,0.0,10.0,0.0,0.379085,0.383482,1.0,1.0,0.0,0.0,0.600000,1.0,0.969780,0.0,1

In [360]:
TEST = df_train[df_train.id.isin(temp)]
df_train = df_train[~df_train.id.isin(temp)]

In [361]:
df_train["demand"] = np.log1p(df_train["demand"])
X = df_train.drop(['demand', 'id'], axis=1)
y = df_train['demand']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [362]:
y_train

0        3.401197
1        4.174387
2        3.496508
3        2.397895
4        2.772589
           ...   
31744    2.197225
31745    2.079442
31746    2.944439
31747    2.302585
31748    1.098612
Name: demand, Length: 30729, dtype: float64

In [363]:
df = TEST.drop(['demand', 'id'], axis=1)

Обучаем случайный лес:

In [364]:
model = RandomForestRegressor(n_estimators=15, max_depth=10, random_state=0, n_jobs=-1).fit(X, y)
y_pred = model.predict(df)
y_pred

array([1.42276295, 1.99562982, 1.16264502, ..., 3.86145811, 2.51574971,
       3.28680821])

In [365]:
y_pred = np.expm1(y_pred)

In [366]:
y_train

0        3.401197
1        4.174387
2        3.496508
3        2.397895
4        2.772589
           ...   
31744    2.197225
31745    2.079442
31746    2.944439
31747    2.302585
31748    1.098612
Name: demand, Length: 30729, dtype: float64

In [367]:
new_df = TEST
new_df['predicted'] = y_pred
#new_df['predicted'] = np.round(y_pred).astype(int)
new_df = new_df[['id', 'predicted']]
new_df.to_csv('out.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [368]:
new_df

,id,predicted
908,908,3.148567
909,909,6.356835
910,910,2.198382
911,911,2.639075
912,912,2.029001
...,...,...
35339,35537,125.836694
35340,35538,55.726544
35341,35539,46.534612
35342,35540,11.375884
